<a href="https://colab.research.google.com/github/Vaibhav543/Applied_AI/blob/main/Predict_Fradulent_Transactions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import required modules
import csv
import sqlite3
import pandas as pd



# Connecting to the geeks database
connection = sqlite3.connect('ml.db')

# Creating a cursor object to execute
# SQL queries on a database table
cursor = connection.cursor()

# Table Definition
Create_Transactions = '''CREATE TABLE Transactions(
        TID INTEGER,
        IdOfOrigStudent INTEGER,
        IdOfDestStudent INTEGER,
        transactionType TEXT,
        hourStep INTEGER,
        transaction_device TEXT,
				comments TEXT);
				'''
Create_Processed_Hacker_Check= '''CREATE TABLE Processed_Hacker_Check(
        TID INTEGER,
        isSuspectedHacker TEXT,
        isHacker TEXT);
				'''
Create_Students = '''CREATE TABLE Students(
        StudentId INTEGER,
        StudentName TEXT,
        StudentAge INTEGER,
        StudentYear INTEGER,
        TransactionFrequency TEXT,
        NumberOfTransactionsTillDate INTEGER,
        AccountType TEXT);
				'''
Create_Sender= '''CREATE TABLE Sender(
        oldBalanceOfOrigStudent INTEGER,
        newBalanceOfOrigStudent INTEGER,
        transactionAmount INTEGER,
        TID INTEGER);
				'''
Create_Receiver= '''CREATE TABLE Receiver(
        oldBalanceOfDestStudent INTEGER,
        newBalanceOfDestStudent INTEGER,
        transactionAmount INTEGER,
        TID INTEGER);
	'''

# Creating the table into our database. Dont Run again if relation is already created.
cursor.execute(Create_Transactions)
#cursor.execute("DROP TABLE Processed_Hacker_Check")
#cursor.execute(Create_Processed_Hacker_Check)
cursor.execute(Create_Students)
cursor.execute(Create_Sender)
cursor.execute(Create_Receiver)

##

# Opening the person-records.csv file
Transactions_file = open('/content/drive/MyDrive/ADS_DataSet/Transactions.csv')
#Processed_Hacker_Check_file = open('/content/Processed_Hacker_Check.csv')
Students_file = open('/content/drive/MyDrive/ADS_DataSet/Students.csv')
Sender_file = open('/content/drive/MyDrive/ADS_DataSet/sender.csv')
Receiver_file = open('/content/drive/MyDrive/ADS_DataSet/receiver.csv')



# Reading the contents of the
# person-records.csv file
Transactions_contents = csv.reader(Transactions_file)
#Processed_Hacker_Check_file_contents = csv.reader(Processed_Hacker_Check_file)
Students_contents = csv.reader(Students_file)
Sender_contents = csv.reader(Sender_file)
Receiver_contents = csv.reader(Receiver_file)


# SQL query to insert data into the
# person table
insert_Transactions = "INSERT INTO Transactions (TID, IdOfOrigStudent,IdOfDestStudent,transactionType,hourStep,transaction_device,comments) VALUES(?,?,?,?,?,?,?) "
insert_Processed_Hacker_Check="INSERT INTO Processed_Hacker_Check (TID, isSuspectedHacker,isHacker) VALUES(?,?,?)"
insert_Students="INSERT INTO Students (StudentId, StudentName,StudentAge,StudentYear,TransactionFrequency,NumberOfTransactionsTillDate,AccountType) VALUES(?,?,?,?,?,?,?)"
insert_Sender="INSERT INTO Sender (oldBalanceOfOrigStudent, newBalanceOfOrigStudent,transactionAmount,TID) VALUES(?,?,?,?)"
insert_Receiver="INSERT INTO Receiver (oldBalanceOfDestStudent, newBalanceOfDestStudent,transactionAmount,TID) VALUES(?,?,?,?)"



# Importing the contents of the file
# into our person table
cursor.executemany(insert_Transactions, Transactions_contents)
#cursor.executemany(insert_Processed_Hacker_Check, Processed_Hacker_Check_file_contents)
cursor.executemany(insert_Students, Students_contents)
cursor.executemany(insert_Sender, Sender_contents)
cursor.executemany(insert_Receiver, Receiver_contents)


# SQL query to retrieve all data from
# the person table To verify that the
# data of the csv file has been successfully
# inserted into the table
select_all = "SELECT * FROM Transactions LIMIT 5"
rows = cursor.execute(select_all).fetchall()

# Output to the console screen
for r in rows:
	print(r)

# Committing the changes
connection.commit()

# closing the database connection
#connection.close()


In [ ]:
import sqlite3
import time
import openpyxl


Create_Processed_Hacker_Check_= '''CREATE TABLE Processed_Hacker_Check(
        TID INTEGER,
        isSuspectedHacker INTEGER,
        isHacker INTEGER);'''




def case_study_1():
    """ Test for case study 1. """
    # Open the database connection and the Excel file
    with sqlite3.connect('ml.db', timeout=20) as db:
        cursor = db.cursor()
        #cursor.execute("DROP TABLE Processed_Hacker_Check")
        cursor.execute(Create_Processed_Hacker_Check_)
        db.commit()

        workbook=openpyxl.load_workbook('/content/drive/MyDrive/ADS_DataSet/Processed_Hacker_Check.xlsx')

        # Access a specific sheet by name
        sheet = workbook['Sheet1']

        insert_Processed_Hacker_Check="INSERT INTO Processed_Hacker_Check (TID, isSuspectedHacker,isHacker) VALUES(?,?,?)"


        # Clear the database
        #db.execute('DELETE FROM albums;')

        # Load data from each Excel sheet into the database
        # Iterate through rows and columns to read data
        for row in sheet.iter_rows(values_only=True):
         try:
              cursor.execute(insert_Processed_Hacker_Check, (str(row[0]),str(row[1]),str(row[2])))
              #cursor.execute(insert_Processed_Hacker_Check, row[0]),str(row[1]),str(row[2])))

         except sqlite3.Error as e:
                print(e)
                db.rollback()

        db.commit()

        # Check if all data have been loaded
        select_stmt = 'SELECT * FROM Processed_Hacker_Check LIMIT 5'
        for row in cursor.execute(select_stmt).fetchall():
            print(row)

case_study_1()


In [ ]:
#SELECT Transactions.TID,IdOfOrigStudent,IdOfDestStudent,hourStep,transaction_device,comments,oldBalanceOfOrigStudent,newBalanceOfOrigStudent,oldBalanceOfDestStudent,newBalanceOfDestStudent	,receiver.transactionAmount ,isSuspectedHacker,isHacker

display = pd.read_sql_query("""
SELECT Transactions.TID,IdOfOrigStudent,IdOfDestStudent,hourStep,transaction_device,comments,oldBalanceOfOrigStudent,newBalanceOfOrigStudent,oldBalanceOfDestStudent,newBalanceOfDestStudent	,receiver.transactionAmount ,isSuspectedHacker,isHacker
FROM Transactions
JOIN sender on sender.TID=Transactions.TID
JOIN receiver on receiver.TID=Transactions.TID
JOIN Processed_Hacker_Check on Processed_Hacker_Check.TID=Transactions.TID""", connection)
display

In [ ]:
display=display.drop("isSuspectedHacker",axis=1)

display.drop(index=0,inplace=True)
#display.drop(display[display['TID'].is].index,inplace=True)
print(display.shape)


In [ ]:

# save the labels into a variable l.
label = display['isHacker']

# Drop the label feature and store the pixel data in d.
data = display.drop("isHacker",axis=1)

print(data.shape)
print(label.shape)


In [ ]:
print(display.dtypes)

In [ ]:

print(display.columns)
print(display['isHacker'].value_counts())

print(display.nunique())


#This is a highly imbalanced dataset as the number of data points for not Hacked Transactions are very less.

In [ ]:
display.isnull().sum()

In [ ]:
display.comments.unique()

In [ ]:
# Data-preprocessing: Standardizing the data
display["transaction_device"].replace(['ATM card', 'Net Banking', 'Check', 'UPI'],[1,2,3,4],inplace=True)

display["comments"].replace(['eatables', 'Purchased groceries', 'food', 'dinner',
       'purchased stationary', 'laundry rent', 'loan',
       'Done at super market.', 'room rent', 'drinks', 'lunch', 'betting',
       'breakfast'],[1,2,3,4,5,6,7,8,9,10,11,12,13],inplace=True)



numeric_data=display.drop(['IdOfOrigStudent','IdOfDestStudent'],axis=1)

print(numeric_data.columns)


In [ ]:
from sklearn.preprocessing import StandardScaler
standardized_data = StandardScaler().fit_transform(numeric_data)
print(standardized_data.shape)

# TSNE for data visulization

from sklearn.manifold import TSNE
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt


model = TSNE(n_components=2,random_state=0,perplexity=2,n_iter=5000)
# configuring the parameteres
# the number of components = 2
# default perplexity = 30
# default learning rate = 200
# default Maximum number of iterations for the optimization = 1000

tsne_data = model.fit_transform(standardized_data)


# creating a new data frame which help us in ploting the result data
tsne_data = np.vstack((tsne_data.T, label)).T
tsne_df = pd.DataFrame(data=tsne_data, columns=("Dim_1", "Dim_2", "label"))

# Ploting the result of tsne
sn.FacetGrid(tsne_df, hue="isHacker", size=6).map(plt.scatter, 'Dim_1', 'Dim_2').add_legend()
plt.show()

In [ ]:
# 2-D Scatter plot with color-coding for each class.
# Here 'sns' corresponds to seaborn.
import seaborn as sn
import matplotlib.pyplot as plt

sn.set_style("whitegrid");
sn.FacetGrid(display, hue="isHacker",height=6) \
   .map(plt.scatter, "transaction_device", "transactionAmount") \
   .add_legend();
plt.show();

In [ ]:
#Pair Plot
sn.set_style("whitegrid");
sn.pairplot(numeric_data, hue="isHacker", height=5);
plt.show()

In [ ]:
#Histogram(1D Scatter Plot)
#PDF is smoothed form of histogram (Percentage of data points that lie between two x values)

g1=sn.FacetGrid(numeric_data,hue="isHacker")
g1.map(sn.distplot,"transaction_device")
g1.add_legend()
plt.show()

In [ ]:
g1=sn.FacetGrid(numeric_data,hue="isHacker")
g1.map(sn.distplot,"transactionAmount")
g1.add_legend()
plt.show()

In [ ]:
#Box-plot with whiskers: It is another method of visualizing the  1-D scatter plot more intuitivey.
sn.boxplot(x='isHacker',y='transactionAmount', data=display)
plt.show()


In [ ]:
#Box Plot
#['ATM card', 'Net Banking', 'Check', 'UPI'],[1,2,3,4]

sn.boxplot(x='isHacker',y='transaction_device', data=numeric_data)
plt.legend(["1-ATM card","2-Net Banking","3-Check","4-UPI"])
plt.show()


In [ ]:
#In Violin Plot, Denser regions of the data are fatter, and sparser ones thinner
#Violin plot

sn.violinplot(x="isHacker", y="transactionAmount", data=display, size=8)
plt.add_legend()
plt.show()